In [2]:
import gc, argparse, sys, os, errno
%pylab inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm import tqdm_notebook as tqdm
import scipy
import sklearn
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings('ignore')
from bokeh.palettes import Category20c

Populating the interactive namespace from numpy and matplotlib


In [3]:
cd ..

/home/chenxupeng/projects/arts_collection


In [5]:
des_arr = np.loadtxt('data/descriptors_num.txt',dtype='str')
sample_info = pd.DataFrame(np.concatenate((np.array([i.split('/')[0] for i in des_arr[:,0] ]).reshape(-1,1),
                                         des_arr ),axis=1))
sample_info['imgname'] = np.array([i.split('/')[1].split('.')[0] for i in sample_info.iloc[:,1]])
sample_info.columns = ['old_class','index','descriptor_num','imgind']
sample_info = sample_info.set_index('index')

In [6]:
ls data/new_category_sum/  #new category

myclass_bamboos.csv*     myclass_glasses.csv*       myclass_religion.csv*
myclass_bronzes.csv*     myclass_handwritings.csv*  myclass_sculptures.csv*
myclass_ceramics.csv*    myclass_impress.csv*       myclass_seals.csv*
myclass_clocks.csv*      myclass_instruments.csv*   myclass_studies.csv*
myclass_embroiders.csv*  myclass_jades.csv*         myclass_tinwares.csv*
myclass_enamels.csv*     myclass_jewels.csv*        myclass_utensils.csv*
myclass_foreign.csv*     myclass_lacquerwares.csv*  myclass_weapons.csv*
myclass_gears.csv*       myclass_paintings.csv*


In [7]:
categories_all = np.array([ i.split('_')[1].split('.')[0] for i in os.listdir('data/new_category_sum/') ])

In [8]:
categories_all

array(['bamboos', 'religion', 'seals', 'handwritings', 'paintings',
       'weapons', 'sculptures', 'instruments', 'jades', 'jewels', 'gears',
       'impress', 'embroiders', 'studies', 'foreign', 'clocks', 'bronzes',
       'tinwares', 'glasses', 'lacquerwares', 'enamels', 'utensils',
       'ceramics'], dtype='<U12')

In [9]:
tmp_tables = {}
for class_name in categories_all:
    tmp_table = pd.read_csv('data/new_category_sum/myclass_'+class_name+'.csv',encoding = "gbk",index_col=0)
    tmp_table['index'] = np.array([class_name+'/'+str(i)+'.jpg' for i in tmp_table.index])

    tmp_table.columns = ['chn_name','new_class','index']
    tmp_table = tmp_table.set_index('index')
    tmp_tables[class_name] = tmp_table

In [10]:
new_table_info = pd.concat(tmp_tables.values(),axis=0)
#new_table_info[np.isin(new_table_info.new_class ,np.array(['镜', '尺', '器', '型']))]['new_class']= 11
#new_table_info[np.isin(new_table_info.new_class ,np.array([111]))]['new_class']= 11

In [11]:
whole_table  = pd.concat((sample_info,new_table_info.loc[sample_info.index]),axis=1)

classes, counts = np.unique(np.array(whole_table.old_class), return_counts=True)
classes = np.argmax(np.array(whole_table.old_class).reshape((-1, 1)) == classes.reshape((1, -1)), axis=1)
counts = np.take(counts, classes)
whole_table['old_class_num'] = classes

In [12]:
whole_table.to_csv('data/whole_sample_info.csv')

In [13]:
np.unique(whole_table['new_class'])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [21]:
whole_table.head()

,old_class,descriptor_num,imgind,chn_name,new_class,old_class_num
index,,,,,,
ceramics/63.jpg,ceramics,1912,63,青釉堆塑谷仓罐,3,2
ceramics/823.jpg,ceramics,946,823,宜兴窑紫砂花口花盆,1,2
ceramics/189.jpg,ceramics,540,189,定窑白釉双系罐,3,2
ceramics/77.jpg,ceramics,340,77,青釉唾壶,4,2
ceramics/837.jpg,ceramics,116,837,仿定窑白釉三牺尊,4,2


In [33]:
for i in np.unique(whole_table.new_class):
    os.makedirs('data/category_new/'+str(i))
for i in np.unique(whole_table.new_class):
    os.makedirs('data/test_new/'+str(i))

FileExistsError: [Errno 17] File exists: 'data/category_new/1'

In [64]:
#pd.read_excel('data/table/dat_'+class_name+'.xlsx',encoding = "gbk",index_col=0)

## split new class train and test
根据新的分类，首先建立`data/category_new`，根据sample info对应copy到对应的文件夹下，注意最后的文件名是旧类的名字

In [40]:
from shutil import copyfile
import shutil
#copyfile(src, dst)

In [23]:
ls data/category/ceramics/63.jpg

data/category/ceramics/63.jpg


In [25]:
ls data/category/ceramics/63.jpg

data/category/ceramics/63.jpg


In [42]:
indexs = whole_table.index
classss = whole_table.new_class
for i in tqdm(range(whole_table.shape[0])):
    tmpind = indexs[i]
    copyfile('data/category/'+tmpind,'data/category_new/'+str(classss[i])+'/'+tmpind.split('/')[0]+'.'+tmpind.split('/')[1])
    

HBox(children=(IntProgress(value=0, max=6516), HTML(value='')))

In [34]:
ls data/category_new/1

bamboos.110.jpg    ceramics.1364.jpg  ceramics.382.jpg  ceramics.810.jpg
bamboos.134.jpg    ceramics.1365.jpg  ceramics.391.jpg  ceramics.811.jpg
bamboos.144.jpg    ceramics.1366.jpg  ceramics.392.jpg  ceramics.822.jpg
bamboos.182.jpg    ceramics.1367.jpg  ceramics.393.jpg  ceramics.823.jpg
bamboos.200.jpg    ceramics.1369.jpg  ceramics.394.jpg  ceramics.824.jpg
bamboos.202.jpg    ceramics.1372.jpg  ceramics.414.jpg  ceramics.840.jpg
bamboos.211.jpg    ceramics.1373.jpg  ceramics.420.jpg  ceramics.841.jpg
bamboos.222.jpg    ceramics.1386.jpg  ceramics.421.jpg  ceramics.842.jpg
bamboos.250.jpg    ceramics.1387.jpg  ceramics.423.jpg  ceramics.843.jpg
bamboos.255.jpg    ceramics.1389.jpg  ceramics.424.jpg  ceramics.844.jpg
bamboos.282.jpg    ceramics.140.jpg   ceramics.425.jpg  ceramics.846.jpg
bamboos.287.jpg    ceramics.144.jpg   ceramics.426.jpg  ceramics.856.jpg
bamboos.69.jpg     ceramics.149.jpg   ceramics.432.jpg  ceramics.873.jpg
bamboos.76.jpg     ceramics.156.jpg   ceramics.439.

In [41]:
for i in os.listdir('data/category_new/'):
    tmpfile = os.listdir('data/category_new/'+i)
    tmplen = len(tmpfile)
    test_ind = np.random.choice(range(tmplen), 
                           int(0.2*tmplen), replace=False)
    for j in np.array(tmpfile)[test_ind]:
        shutil.move('data/category_new/'+i+'/'+j,'data/test_new/'+i+'/'+j)

In [16]:
test_ind = np.random.choice(range(whole_table.shape[0]), 
                           int(0.2*whole_table.shape[0]), replace=False)
for i in os.listdir('data/train/'):
    os.makedirs('data/test/'+i)
    tmp_select_df = image_size_df.iloc[test_ind]
for i in range(tmp_select_df.shape[0]):
    shutil.move('data/train/'+tmp_select_df.iloc[i]['category']
                +'/'+tmp_select_df.iloc[i]['img_name'],'data/test/'+tmp_select_df.iloc[i]['category']
                +'/'+tmp_select_df.iloc[i]['img_name'])

FileNotFoundError: [Errno 2] No such file or directory: 'data/train/'